# Spatial Proteomics analysis (MACSima)

## 1. Read in the data

In [ ]:
%load_ext autoreload
%autoreload 2

### 1.1 MACSima reader

In [ ]:
import harpy as hp
from spatialdata_io import macsima

input_path =  r"D:\example_data_MACSima\REAscreen_IO_CRC\REAscreen_IO_CRC" # Set input path to folder containing MACSima images
output_path = r"D:\example_data_MACSima\REAscreen_IO_CRC\output_harpy" # Set a path to save the output from Harpy

In [ ]:
# Read in MACSima data using the spatialdata-io macsima reader
sdata = macsima(
    path = input_path,
    split_threshold_nuclei_channel=1,
)

In [ ]:
# We need to make sure the SpatialData opject is backed to disk (which is not included in the reader function)
import os
from spatialdata import read_zarr

# Write the SpatialData object to disk
sdata.write(os.path.join(output_path, "sdata.zarr"))

# Read back in
sdata = read_zarr(os.path.join(output_path, "sdata.zarr"))
sdata

In [ ]:
# We will make a global coordinate system
from spatialdata.transformations import Identity
from spatialdata.transformations import get_transformation, set_transformation

transformations=get_transformation(sdata["REAscreen_IO_CRC_image"], get_all=True)
transformations["global"] = Identity()
set_transformation(sdata["REAscreen_IO_CRC_image"], transformation=transformations, set_all=True, write_to_sdata=sdata)

transformations=get_transformation(sdata["REAscreen_IO_CRC_nuclei_image"], get_all=True)
transformations["global"] = Identity()
set_transformation(sdata["REAscreen_IO_CRC_nuclei_image"], transformation=transformations, set_all=True, write_to_sdata=sdata)

# Inspect the sdata
sdata

In [ ]:
# Inspect the transformations associated with the "REAscreen_IO_CRC_image" image layer
get_transformation(
    sdata["REAscreen_IO_CRC_image"],
    get_all=True,
)

In [ ]:
# Remove table layers from memory and disk
import shutil

del sdata.tables['REAscreen_IO_CRC_nuclei_table']
del sdata.tables['REAscreen_IO_CRC_table']

shutil.rmtree(sdata.path / 'tables')

In [ ]:
# Explore the sdata interactively
from napari_spatialdata import Interactive

# Interactive(sdata)

In [ ]:
# List all channel names
channels = sdata["REAscreen_IO_CRC_image"]["scale0"]["image"].c.data
channels

In [ ]:
# List all nuclei channel names
nuclei_channels = sdata["REAscreen_IO_CRC_nuclei_image"]["scale0"]["image"].c.data
nuclei_channels

### 1.2 QC

In [ ]:
# Plot images without normalization (works for bright images such as DAPI)
import spatialdata_plot

ax=sdata.pl.render_images(
    element="REAscreen_IO_CRC_image",
    channel="DAPI (1)",
    cmap="grey",
    scale="scale4",
    norm=None,
).pl.show(
    coordinate_systems="global",
    figsize=(6,6), 
    dpi=100, 
    colorbar=False,
)

In [ ]:
# Plot images with normalization (to adjust the minimum and maximum visualization values)
from matplotlib.colors import Normalize

vmax = 1000
vmin = 0
norm = Normalize(vmax=vmax, vmin=vmin, clip=True)

ax=sdata.pl.render_images(
    "REAscreen_IO_CRC_image",
    channel="CD45",
    cmap="grey",
    scale="scale4",
    norm=norm,
).pl.show(
    coordinate_systems="global",
    figsize=(6,6), 
    dpi=100, 
    colorbar=False,
)

In [ ]:
# We can create histograms for each channel using the following function:
hp.pl.histogram(
    sdata,
    img_layer="REAscreen_IO_CRC_image",
    channel="CD45",
    bins=100,
    fig_kwargs={
        "figsize": (4, 4),
    },
)

In [ ]:
# To plot the histograms for all channels:
import matplotlib.pyplot as plt
import math
import numpy as np

# Grid config
n_cols = 3
n_rows = math.ceil(len(channels) / n_cols)

# Specify channels to plot
hist_channels = channels

# Create subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 3))
axes = axes.flatten()  # Flatten to 1D array for easy indexing

# Get minimum and maximum values for img dtype
img_dtype = sdata.images["REAscreen_IO_CRC_image"]["scale0"]['image'].dtype
img_min = np.iinfo(img_dtype).min
img_max = np.iinfo(img_dtype).max

# Plot each histogram in its own subplot
for i, channel in enumerate(hist_channels):
    ax = axes[i]
    hp.pl.histogram(
        sdata,
        img_layer="REAscreen_IO_CRC_image",
        channel=channel,
        range=(img_min, img_max/2),
        bins=100,
        ax=ax,  # Plot into specific subplot
    )
    ax.set_title(channel)

# Turn off any unused axes
for j in range(i + 1, len(axes)):
    axes[j].axis("off")

plt.tight_layout()
plt.show()

## 2. QC and annotations of ROIs and artifacts in Qupath
Qupath is open source software for bioimage analysis. It has some really nice tools for annotating regions of interest and artifacts in images. We will use these for our dataset and add the results to the SpatialData object.

1. If necessary, install QUpath (v.0.5.1) using this [link](https://qupath.github.io/).
2. Open Qupath and click `Create project` in the top left corner.
3. Create a new empty folder for the project and click `Select Folder`.
4. You would normally add images to a project by going to `Add images` and providing the image paths. However, this would result in the images being added separately, and we would prefer a merged image containing all channels. To do this, we will use the `read_multichannel_qupath.groovy` script. To run it, go to `Automate`, `Script Editor`, `File`, `Open`, and select the script. In the Qupath script editor, change the folderPath argument to the folder containing the MACSima images and click `Run`. This should add all images to the project as a single image.
5. Explore the image by going to `Brightness & contrast`, select the channels you want to visualize and adjust the settings to improve the visualization.

### 2.1 QC image channels

<b>Excercise</b>:
- Create a python list of which channels you would consider successful and a list for the channels you believe failed. Since we're not experts in this tissue, we'll aim for removing the channels with obvious flaws (note that this is subjective and very dependent on how well you know the underlying biology and expected staining patterns for the biomarkers).

<details> 
<summary>Click to reveal the solution</summary>

```python
successful_channels = [
    'DAPI (1)',
    'Bcl 2', # Clearly contains acquisition bleaching artifacts, but we can annotate these regions to remove them
    'CD8a',
    'FoxP3',
    'CD138',
    'CD15',
    'CD45',
    'Collagen I',
    'CD274',
    'PCNA',
    'CD163',
    'CD56',
    'CD11b',
    'CD45RB',
    'CD14',
    'CD107a',
    'Ki 67',
    'CD4',
    'CD107b',
    'CD31',
    'CD38',
    'CD66b',
    'Cleaved PARP1',
    'Mast Cell Tryptase',
    'CD20 Cytoplasmic',
    'Podoplanin',
    'CD16',
    'CD1c',
    'Actin',
    'CD68',
    'Cytokeratin 20',
    'CD79a',
    'CD45RO',
    'Vimentin',
    'CD11c',
    'CD3',
    'CD324',
    'HLA ABC',
    'CD57',
    'CD204',
    'MART 1',
    'Melanocyte PMEL',
    'Cytokeratin 19',
    'Cytokeratin',
    'HLA DR',
    'beta Catenin',
    'beta Actin',
    'CD326',
    'Calponin',
    'DAPI (2)',
]

failed_channels = [
    'CD209', # Uneven illumination
    'CD279', # Uneven illumination
    'CD44', # Uneven illumination
    'p53', # Uneven illumination
    'Cytokeratin 7', # Uneven illumination
    'CD223', # Very bright (saturating) artifact
    'Cytokeratin 14', # Uneven illumination
    'CD45RA', # Uneven illumination
    'CD66b', # Very bright (saturating) artifact
    'CD134', # Uneven illumination
    'Cytokeratin 5', # Uneven illumination
    'Cytokeratin 8', # Uneven illumination
    'Cytokeratin HMW', # Uneven illumination
]

In [ ]:
successful_channels = [
    'DAPI (1)',
    'Bcl 2', # Clearly contains acquisition bleaching artifacts, but we can annotate these regions to remove them
    'CD8a',
    'FoxP3',
    'CD138',
    'CD15',
    'CD45',
    'Collagen I',
    'CD274',
    'PCNA',
    'CD163',
    'CD56',
    'CD11b',
    'CD45RB',
    'CD14',
    'CD107a',
    'Ki 67',
    'CD4',
    'CD107b',
    'CD31',
    'CD38',
    'CD66b',
    'Cleaved PARP1',
    'Mast Cell Tryptase',
    'CD20 Cytoplasmic',
    'Podoplanin',
    'CD16',
    'CD1c',
    'Actin',
    'CD68',
    'Cytokeratin 20',
    'CD79a',
    'CD45RO',
    'Vimentin',
    'CD11c',
    'CD3',
    'CD324',
    'HLA ABC',
    'CD57',
    'CD204',
    'MART 1',
    'Melanocyte PMEL',
    'Cytokeratin 19',
    'Cytokeratin',
    'HLA DR',
    'beta Catenin',
    'beta Actin',
    'CD326',
    'Calponin',
    'DAPI (2)',
]

failed_channels = [
    'CD209', # Uneven illumination
    'CD279', # Uneven illumination
    'CD44', # Uneven illumination
    'p53', # Uneven illumination
    'Cytokeratin 7', # Uneven illumination
    'CD223', # Very bright (saturating) artifact
    'Cytokeratin 14', # Uneven illumination
    'CD45RA', # Uneven illumination
    'CD66b', # Very bright (saturating) artifact
    'CD134', # Uneven illumination
    'Cytokeratin 5', # Uneven illumination
    'Cytokeratin 8', # Uneven illumination
    'Cytokeratin HMW', # Uneven illumination
]

### 2.2 Annotate artifacts

1. Open the `Bcl 2 (C3)` channel and annotate the acquisition bleaching regions. Since these are straight lines, you can use the rectangle annotation to annotate all the segments and subsequently merge the annotation by going to `Objects`, `Annotations...`, `Merge selected`. Rename the merged annotation to `acquisition_bleaching` by right-clicking on the annotation in the `Annotations` tab and going to `Set properties`.

2. Open the `DAPI (C1)` channel. In other datasets, we may find some out-of-focus regions, tissue-folds, etc that we could annotate. In this dataset, there is a small very bright region at the top right of the tissue. Annotate it using either the polygon annotation tool or the magic brush, merge any annotation objects you create and rename the merged object `bright_artifact`.

3. Export the annotations. Select all artifact annotations and go to `File`, `Export objects as GeoJSON` and set to `Selected objects` and `Compression` to None. Click `OK` and set the file name to `artifacts.geojson`.

### 2.3 Annotate ROIs

1. Now, pick a region that looks of interest to you, annotate it and label it `ROI`. Select it and export the GeoJSON as `ROI.geojson.`

### 2.4 Add annotations to sdata

In [ ]:
import geopandas as gpd

path_to_artifacts_GeoJSON = r'd:\example_data_MACSima\REAscreen_IO_CRC\Qupath\artifacts.geojson'
path_to_ROI_GeoJSON = r'd:\example_data_MACSima\REAscreen_IO_CRC\Qupath\ROI.geojson'

gdf_artifacts = gpd.read_file(path_to_artifacts_GeoJSON)
sdata = hp.sh.add_shapes_layer(sdata, input=gdf_artifacts, output_layer='artifacts', overwrite=True)

gdf_ROI = gpd.read_file(path_to_ROI_GeoJSON)
sdata = hp.sh.add_shapes_layer(sdata, input=gdf_ROI, output_layer='ROI', overwrite=True)

sdata

In [ ]:
# Inspect artifacts shapes layer
sdata.shapes['artifacts']

In [ ]:
# Plot artifacts layer
hp.pl.plot_shapes(
    sdata, 
    img_layer='REAscreen_IO_CRC_image', 
    channel='DAPI (1)', 
    shapes_layer='artifacts',
)

In [ ]:
# Inspect ROI shapes layer
sdata.shapes['ROI']

In [ ]:
# Plot ROI layer
hp.pl.plot_shapes(
    sdata, 
    img_layer='REAscreen_IO_CRC_image', 
    channel='DAPI (1)', 
    shapes_layer='ROI',
)

## 3. Segmentation 

In [ ]:
# First, we will create a cropped version of the SpatialData object to speed up computation
x_min, x_max, y_min, y_max = 7000, 10000, 2000, 5000

sdata_crop = sdata.query.bounding_box(
    min_coordinate=[x_min, y_min], max_coordinate=[x_max, y_max], axes=("x", "y"), target_coordinate_system="global"
)

# Crop shapes layers
from shapely.geometry import box
bbox = box(x_min, y_min, x_max, y_max)

for shapes_layer in sdata_crop.shapes.keys():
    gdf = sdata_crop.shapes[shapes_layer]
    gdf_cropped = gdf.clip(bbox)
    sdata_crop = hp.sh.add_shapes_layer(sdata_crop, input=gdf, output_layer=shapes_layer, overwrite=True)
    
# Write the SpatialData object to disk
sdata_crop.write(os.path.join(output_path, "sdata_crop.zarr"))

# Read back in
sdata_crop = read_zarr(os.path.join(output_path, "sdata_crop.zarr"))
sdata_crop

### 3.1 Create subset of segmentation channels

In [ ]:
# First, we create a subset of channels that delineate the nucleus and cell borders for a wide variety of cell types
segmentation_channels = ["DAPI (1)", "CD8a", "CD15", "CD45", "PCNA", "CD4", "CD38", "Mast Cell Tryptase", "Vimentin", "CD11c", "CD3", "HLA ABC", "CD326"]

# We create a dictionary containing the index for every channel
channel_indexes = {channel: index for index, channel in enumerate(channels)}

# Now, we can get the indexes for the segmentation channels
segmentation_channels_indexes = [channel_indexes[channel] for channel in segmentation_channels]
segmentation_channels_indexes

In [ ]:
# Subset image based on segmentation channels
arr = sdata_crop.images["REAscreen_IO_CRC_image"]["scale0"]["image"].data[segmentation_channels_indexes]

# Get transformations
from spatialdata.transformations import get_transformation
transformations = get_transformation(sdata_crop.images["REAscreen_IO_CRC_image"], get_all=True)

# Add new image layer
sdata_crop = hp.im.add_image_layer(
    sdata_crop,
    arr=arr,
    output_layer="REAscreen_IO_CRC_image_segmentation_channels",
    transformations=transformations,
    c_coords=segmentation_channels,
    overwrite=True,
)

### 3.2 Image preprocessing

In [ ]:
# Normalize images based on lower and upper percentile
sdata_crop = hp.im.normalize(
    sdata_crop, 
    img_layer = "REAscreen_IO_CRC_image_segmentation_channels",
    output_layer = "REAscreen_IO_CRC_image_segmentation_channels_norm",
    q_min = 20.0,  
    q_max = 99.9,
    overwrite=True
)

# hp.im.normalize gives us an image with dtype float32, but we want uint16 since clahe does not support float32 so we'll change the dtype
img_float32 = sdata_crop.images['REAscreen_IO_CRC_image_segmentation_channels_norm'] # Values fall between 0 and 1
img_uint16 = (img_float32 * 65535).clip(0, 65535).astype(np.uint16) # Values fall between 0 and 65535

transformations = get_transformation(sdata_crop["REAscreen_IO_CRC_image_segmentation_channels_norm"], get_all=True)

sdata_crop = hp.im.add_image_layer(
    sdata_crop,
    arr=img_uint16.data,
    output_layer="REAscreen_IO_CRC_image_segmentation_channels_norm",
    transformations=transformations,
    c_coords=segmentation_channels,
    overwrite=True,
)

In [ ]:
# Plot the normalized images
hp.pl.plot_image(
    sdata_crop,
    img_layer = ["REAscreen_IO_CRC_image_segmentation_channels", "REAscreen_IO_CRC_image_segmentation_channels_norm"],
    figsize = (20, 100),
)

In [ ]:
# Perform min max filtering
sdata_crop = hp.im.min_max_filtering(
    sdata_crop,
    img_layer = "REAscreen_IO_CRC_image_segmentation_channels_norm",
    output_layer = "REAscreen_IO_CRC_image_segmentation_channels_minmax",
    size_min_max_filter = 55,
    overwrite = True,
)

In [ ]:
# Plot the min max filtered images
hp.pl.plot_image(
    sdata_crop,
    img_layer = ["REAscreen_IO_CRC_image_segmentation_channels_norm", "REAscreen_IO_CRC_image_segmentation_channels_minmax"],
    figsize = (20, 100),
)

In [ ]:
# Perform contrast enhancement using CLAHE
sdata_crop = hp.im.enhance_contrast(
    sdata_crop,
    img_layer = "REAscreen_IO_CRC_image_segmentation_channels_minmax",
    output_layer = "REAscreen_IO_CRC_image_segmentation_channels_clahe",
    contrast_clip = 3.5,
    chunks = 2048,
    overwrite = True
)

In [ ]:
# Plot the contrast enhanced images
hp.pl.plot_image(
    sdata_crop,
    img_layer = ["REAscreen_IO_CRC_image_segmentation_channels_minmax", "REAscreen_IO_CRC_image_segmentation_channels_clahe"],
    figsize = (20, 100),
)

### 3.3 Nuclei and cell segmentation

In [ ]:
# We need to download and unzip the Instanseg model
import os
import requests
import zipfile
import tempfile

OUTPUT_DIR =  tempfile.gettempdir()

def download_and_unzip(url, extract_to):
    try:
        os.makedirs(extract_to, exist_ok=False)
    except FileExistsError:
        print("Model already downloaded.")
        return
    local_zip_path = os.path.join(extract_to, 'downloaded.zip')
    print("Downloading...")
    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(local_zip_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    print("Unzipping...")
    with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
    os.remove(local_zip_path)
    print(f"Done! Files extracted to: {extract_to}")

url = "https://github.com/instanseg/instanseg/releases/download/instanseg_models_v0.1.0/fluorescence_nuclei_and_cells.zip"
target_path = os.path.join(OUTPUT_DIR, "fluorescence_nuclei_and_cells" )
download_and_unzip(url, target_path)

In [ ]:
from dask.distributed import Client, LocalCluster

# Create a local Dask cluster
cluster = LocalCluster(
    n_workers=4,             # Number of worker processes. Possible to increase to more workers, depending on available memory/cores
    threads_per_worker=1,    # Number of threads per worker
    memory_limit="32GB",     # Memory limit per worker
)

# Connect a Client to the cluster
client = Client(cluster)

# Print the Dask dashboard link
print(client.dashboard_link)

In [ ]:
# Nuclei segmentation using Instanseg (only on DAPI channel)
import torch
from instanseg import InstanSeg

path_model = os.path.join(target_path, "instanseg.pt")

instanseg_fluorescence = torch.load(path_model, weights_only=False)
instanseg_fluorescence = InstanSeg(model_type=instanseg_fluorescence, device="cpu")

arr = sdata_crop.images["REAscreen_IO_CRC_image_segmentation_channels_clahe"].data[[0]] # DAPI (1) has index 0

from spatialdata.transformations import get_transformation
transformations = get_transformation(sdata_crop.images["REAscreen_IO_CRC_image_segmentation_channels_clahe"], get_all=True)

sdata_crop = hp.im.add_image_layer(
    sdata_crop,
    arr=arr,
    output_layer="REAscreen_IO_CRC_image_segmentation_channels_clahe_DAPI",
    transformations=transformations,
    c_coords=segmentation_channels[0],
    overwrite=True,
)

sdata_crop = hp.im.segment(
    sdata_crop,
    img_layer="REAscreen_IO_CRC_image_segmentation_channels_clahe_DAPI",
    output_labels_layer=["labels_nuclei_instanseg"],
    output_shapes_layer=["shapes_nuclei_instanseg"],
    labels_layer_align=None,
    depth=50,
    chunks=2048,
    model=hp.im.instanseg_callable,
    # parameters passed to hp.im.instanseg_callable
    output="nuclei",
    device="cpu",
    instanseg_model=path_model,  # load it in every worker, because torchscript model is not serializable
    pixel_size = 0.17, # pixel size in μm. Setting this correctly has a huge impact on the quality of the results.
    iou=True,
    trim=False,
    to_coordinate_system="global",
    overwrite=True,
)

In [ ]:
# Nuclei segmentation using Cellpose (only on DAPI channel)
sdata_crop = hp.im.segment(
    sdata_crop,
    img_layer="REAscreen_IO_CRC_image_segmentation_channels_clahe_DAPI",
    chunks=2048,
    depth=200,
    model=hp.im.cellpose_callable,
    # parameters that will be passed to the callable _cellpose:
    pretrained_model="nuclei",
    device="cpu",
    diameter=45,
    flow_threshold=0.9,
    cellprob_threshold=-3,
    output_labels_layer="labels_nuclei_cellpose",
    output_shapes_layer="shapes_nuclei_cellpose",
    overwrite=True,
)

In [ ]:
# Cell segmentation using Instanseg (on all channels)
sdata_crop = hp.im.segment(
    sdata_crop,
    img_layer="REAscreen_IO_CRC_image_segmentation_channels_clahe",
    output_labels_layer=["labels_cells_instanseg"],
    output_shapes_layer=["shapes_cells_instanseg"],
    labels_layer_align=None,
    depth=50,
    chunks=2048,
    model=hp.im.instanseg_callable,
    # parameters passed to hp.im.instanseg_callable
    output="cells",
    device="cpu",
    instanseg_model=path_model,  # load it in every worker, because torchscript model is not serializable
    pixel_size = 0.17, # pixel size in μm. Setting this correctly has a huge impact on the quality of the results.
    iou=True,
    trim=False,
    to_coordinate_system="global",
    overwrite=True,
)

client.close()

In [ ]:
hp.pl.plot_shapes(
    sdata_crop,
    img_layer="REAscreen_IO_CRC_image_segmentation_channels_clahe",
    shapes_layer=["shapes_nuclei_instanseg", "shapes_nuclei_cellpose", "shapes_cells_instanseg"],
    channel="DAPI (1)",
    alpha=1,
    linewidth=1,
    figsize=(30,10),
    vmin_img=0,
    vmax_img=65535,
)

### 3.4 Filter segmentation masks based on size and artifacts

In [ ]:
# Filter out nuclei based on size
pixel_size_um = 0.17
square_pixel_size_um2 = pixel_size_um**2

min_area_um2 = 10 # Minimum size of segmentation objects (in square micrometer).
max_area_um2 = 200 # Maximum size of segmentation objects (in square micrometer).

print(f"Total number of segmentation objects: {len(sdata_crop.shapes['shapes_cells_instanseg'])}")
print(f"Filtering out segmentation objects smaller than {min_area_um2} µm² and larger than {max_area_um2} µm².")
gdf = sdata_crop.shapes['shapes_cells_instanseg'][
    (sdata_crop.shapes['shapes_cells_instanseg'].area >= (min_area_um2/square_pixel_size_um2)) &
    (sdata_crop.shapes['shapes_cells_instanseg'].area <= (max_area_um2/square_pixel_size_um2))
]
cells_removed_size = len(sdata_crop.shapes['shapes_cells_instanseg']) - len(gdf)
print(f"Number of segmentation objects removed based on size: {cells_removed_size}")

sdata_crop = hp.sh.add_shapes_layer(
    sdata_crop,
    input = gdf,
    output_layer = 'shapes_cells_instanseg_size_filtered',
    overwrite = True,
)

sdata_crop = hp.im.rasterize(
    sdata_crop,
    shapes_layer = 'shapes_cells_instanseg_size_filtered',
    output_layer = 'labels_cells_instanseg_size_filtered',
    chunks = 5000,
    overwrite = True,
)

In [ ]:
# Plot artifacts layer
hp.pl.plot_shapes(
    sdata_crop, 
    img_layer='REAscreen_IO_CRC_image_segmentation_channels_clahe', 
    channel='DAPI (1)', 
    shapes_layer='artifacts',
)

In [ ]:
# Filter out segmentation masks in artifacts
print(f'Filtering out segmentation objects in artifacts annotations.')
cells_gdf = sdata_crop.shapes['shapes_cells_instanseg_size_filtered']
artifacts_gdf = sdata_crop.shapes['artifacts']

# Identify cells whose geometry intersects the artifacts
cells_gdf['overlaps_artifact'] = cells_gdf.geometry.intersects(artifacts_gdf.unary_union)
filtered_cells_gdf = cells_gdf[~cells_gdf['overlaps_artifact']].copy()
filtered_cells_gdf.drop(columns=['overlaps_artifact'], inplace=True)

cells_removed_artifacts = len(cells_gdf) - len(filtered_cells_gdf)
print(f"Number of segmentation objects removed from artifact annotations: {cells_removed_artifacts}")

sdata_crop = hp.sh.add_shapes_layer(
    sdata_crop, 
    input = filtered_cells_gdf,
    output_layer = 'shapes_cells_instanseg_artifacts_filtered',
    overwrite = True
)

sdata_crop = hp.im.rasterize(
    sdata_crop,
    shapes_layer = 'shapes_cells_instanseg_artifacts_filtered',
    output_layer = 'labels_cells_instanseg_artifacts_filtered',
    chunks = 5000,
    overwrite = True,
)

# FIXME

In [ ]:
hp.pl.plot_shapes(
    sdata_crop,
    img_layer="REAscreen_IO_CRC_image_segmentation_channels_clahe",
    shapes_layer=["shapes_cells_instanseg", "shapes_cells_instanseg_size_filtered", "shapes_cells_instanseg_artifacts_filtered"],
    channel="DAPI (1)",
    alpha=1,
    linewidth=1,
    figsize=(30,10),
    vmin_img=0,
    vmax_img=65535,
)

### 3.5 Align nucleus and cell segmentations

In [ ]:
# Align the nucleus and cell segmentations (we'll use the cellpose nucleus segmentation since it outperformed Instanseg)
sdata_crop=hp.im.align_labels_layers( 
    sdata_crop,
    labels_layer_1="labels_nuclei_cellpose",
    labels_layer_2="labels_cells_instanseg_artifacts_filtered",
    output_labels_layer="labels_nuclei_cellpose_aligned",
    output_shapes_layer="shapes_nuclei_cellpose_aligned",
    chunks=2048,
    depth=300, # Make sure to set this to a number larger than the approx. cell size to avoid chunking effects.
    overwrite=True,
)

# This function aligns two label layers by examining the labels in labels_layer_1 and identifying their maximum overlap with labels in labels_layer_2.
# It then updates the labels of labels_layer_1 (i.e. "labels_nuclei_cellpose").
# If a label from labels_layer_1 has no overlap with a label from label_layer_2, the label in labels_layer_1 is set to zero (i.e. background).
# If two labels from labels_layer_1 overlap with the same label from labels_layer_2, they are effectively merged into the same new label.

In [ ]:
# Sanity check
hp.pl.sanity(
    sdata_crop,
    img_layer="REAscreen_IO_CRC_image_segmentation_channels_clahe",
    shapes_layer="shapes_nuclei_cellpose",
    points_layer=None,
    crd=[7000, 7400, 3500, 3900],
    plot_cell_number=True,
    figsize=(8,8)
)

hp.pl.sanity(
    sdata_crop,
    img_layer="REAscreen_IO_CRC_image_segmentation_channels_clahe",
    shapes_layer="shapes_cells_instanseg_artifacts_filtered",
    points_layer=None,
    crd=[7000, 7400, 3500, 3900],
    plot_cell_number=True,
    figsize=(8,8),
)

hp.pl.sanity(
    sdata_crop,
    img_layer="REAscreen_IO_CRC_image_segmentation_channels_clahe",
    shapes_layer="shapes_nuclei_cellpose_aligned",
    points_layer=None,
    crd=[7000, 7400, 3500, 3900],
    plot_cell_number=True,
    figsize=(8,8)
)

## 4. Cell properties

### 4.1 Cell intensities

In [ ]:
# Calculate intensities for cell segmentations.
marker_channels = [channel for channel in successful_channels if channel not in nuclei_channels]

sdata_crop = hp.tb.allocate_intensity( 
    sdata_crop,
    img_layer="REAscreen_IO_CRC_image",
    labels_layer="labels_cells_instanseg_artifacts_filtered",
    output_layer="table_cells",
    channels=marker_channels,
    mode="sum", # When set to "sum", the total intensity for each channel will be added to `.X` of the table layer; if set to `"mean"`, it calculates the average intensity per channel.
    # obs_stats=["sum", "mean", "count", "var", "kurtosis", "skew", "max", "min"] # Stats that will be added to `.obs` of `output_layer`
    chunks = 2048,
    to_coordinate_system="global",
    overwrite=True,
)

In [ ]:
# Inspect new table layer
sdata_crop.tables["table_cells"]

In [ ]:
# Inspect X
sdata_crop.tables["table_cells"].to_df()

In [ ]:
# Inspect obs
sdata_crop.tables["table_cells"].obs

In [ ]:
# Calculate intensities for nucleus segmentations.
sdata_crop = hp.tb.allocate_intensity( 
    sdata_crop,
    img_layer="REAscreen_IO_CRC_image",
    labels_layer="labels_nuclei_cellpose_aligned",
    output_layer="table_nucleus",
    channels=marker_channels,
    mode="sum", # When set to "sum", the total intensity for each channel will be added to `.X` of the table layer; if set to `"mean"`, it calculates the average intensity per channel.
    # obs_stats=["sum", "mean", "count", "var", "kurtosis", "skew", "max", "min"] # Stats that will be added to `.obs` of `output_layer`
    chunks = 2048,
    to_coordinate_system="global",
    overwrite=True,
)

### 4.2 Morphological properties

In [ ]:
# Add morphological properties for cell
sdata_crop = hp.tb.add_regionprop_features(
    sdata_crop, 
    labels_layer='labels_cells_instanseg_artifacts_filtered', 
    table_layer='table_cells'
)

# area: Area of the segmentation object (in pixels).
# eccentricity: A measure for how much the segmentation object deviates from being circular. Values are between 0-1 (a perfect circle would have value 0).
# major_axis_length: The length of the longest axis of the segmentation object (in pixels).
# minor_axis_length: The length of the shortest axis of the segmentation object (in pixels).
# perimeter: The perimeter of the segmentation object (in pixels).
# centroid_x: x-coordinate of the centroid of the segmentation object (in pixels)
# centroid_y: y-coordinate of the centroid of the segmentation object (in pixels)
# convex_area: Area of the convex hull image, which is the smallest convex polygon that encloses the region (in pixels).
# equivalent_diameter: The diameter of a circle with the same area as the segmentation object (in pixels).
# _major_minor_axis_ratio: The ratio of the major_axis_length over the minor_axis_length. 
# _perim_square_over_area: The ratio of the square of the perimeter over the area.
# _major_axis_equiv_diam: major_axis_length / equivalent_diameter.
# _convex_hull_resid: (convex_area - area) / convex_area.
# _centroid_dif: the normalized euclidean distance between the segmentation object centroid and the corresponding convex hull centroid.
# NOTE: While everything is calculated in pixels, later in this notebook, we will convert those values into micrometers

In [ ]:
# Inspect obs
sdata_crop.tables["table_cells"].obs

In [ ]:
# Add morphological properties for nuclei
sdata_crop = hp.tb.add_regionprop_features(
    sdata_crop, 
    labels_layer='labels_nuclei_cellpose_aligned', 
    table_layer='table_nucleus'
)

In [ ]:
# Inspect obs
sdata_crop.tables["table_nucleus"].obs

### 4.3 Clean and merge nucleus and cell tables

In [ ]:
# Clean table layers
import pandas as pd
from harpy.utils._keys import _INSTANCE_KEY, _REGION_KEY
import shutil
import re

pixel_size_um = 0.17
square_pixel_size_um2 = pixel_size_um**2

def clean_table_layer(sdata, table_layer, suffix, channels, pixel_size_um, square_pixel_size_um2):   
    # Create copy of table layer
    adata = sdata.tables[table_layer].copy()
    
    # Add columns with total intensities to obs
    intensity_df = adata.to_df()
    adata.obs = pd.merge(adata.obs, intensity_df, left_index=True, right_index=True, how='left')

    # Correct intensities for area
    for channel in channels:
        adata.obs[f'{channel}_average_intensity'] = adata.obs[f'{channel}'] / adata.obs['area']
            
    # Drop columns with total intensities
    adata.obs = adata.obs.drop(columns=channels)

    # Sanitize column names (obs column names must contain only alphanumeric characters, underscores, dots and hyphens)
    def sanitize_column_names(df):
        df.columns = [
            re.sub(r"[^\w\.-]", "_", col)  # Only allow [a-zA-Z0-9_.-], replace others with underscore
            for col in df.columns
        ]
        return df

    adata.obs = sanitize_column_names(adata.obs)

    # Clean sdata.table.obs and convert pixel measurements to micrometer
    area_columns = ["area", "convex_area"]
    length_columns = ["major_axis_length", "minor_axis_length", "perimeter", "equivalent_diameter"]

    adata.obs = (
        adata.obs
        # Convert area columns from pixels squared to micrometers squared
        .apply(lambda x: x * square_pixel_size_um2 if x.name in area_columns else x)
        # Convert length columns from pixels to micrometers
        .apply(lambda x: x * pixel_size_um if x.name in length_columns else x)
        # Strip underscores from column names
        .rename(columns=lambda x: x.lstrip('_'))
        # Change names of centroid columns
        .rename(columns={
                'centroid-0': 'centroid_y',
                'centroid-1': 'centroid_x',
        })
        # Add suffixes to columns
        .rename(columns=lambda x: f'{x}_{suffix}' if x not in [_INSTANCE_KEY, _REGION_KEY] else x)
    )

    return adata   

adata_nucleus = clean_table_layer(sdata_crop, 'table_nucleus', 'nucleus', marker_channels, pixel_size_um, square_pixel_size_um2)
adata_cells = clean_table_layer(sdata_crop, 'table_cells', 'cells', marker_channels, pixel_size_um, square_pixel_size_um2)

# Merge adata
obs_nucleus = adata_nucleus.obs.drop(columns=[_REGION_KEY]).reset_index(drop=True) # Remove region key
cells_obs = adata_cells.obs.reset_index()
adata_cells.obs = pd.merge(cells_obs, obs_nucleus, on='cell_ID', how='left', suffixes=('', '_nucleus'))

# Add column that checks whether the cell has a nucleus
adata_cells.obs['contains_nucleus'] = ~adata_cells.obs['centroid_x_nucleus'].isna()

# Add merged table back to sdata
sdata_crop = hp.tb.add_table_layer(
    sdata_crop,
    adata=adata_cells,
    output_layer='table_intensities',
    region=adata_cells.obs[_REGION_KEY].cat.categories.to_list(),
    overwrite=True,
)

# Remove table layers from memory and disk
del sdata_crop.tables['table_nucleus']
del sdata_crop.tables['table_cells']

shutil.rmtree(sdata_crop.path / 'tables' / 'table_nucleus')
shutil.rmtree(sdata_crop.path / 'tables' / 'table_cells')

In [ ]:
# Inspect table
sdata_crop.tables['table_intensities']

In [ ]:
# Inspect table obs
sdata_crop.tables['table_intensities'].obs

In [ ]:
# Plot which cells have nuclei
hp.pl.plot_shapes( 
    sdata_crop,
    img_layer="REAscreen_IO_CRC_image",
    table_layer="table_intensities",
    shapes_layer="shapes_cells_instanseg_artifacts_filtered",
    column="contains_nucleus",
    cmap='Set1',
    channel="DAPI (1)",
    linewidth=0.2,
    alpha=0.7,
    figsize=(8,8),
    to_coordinate_system="global",
)

In [ ]:
# Plot cell density using matplotlib
import matplotlib.pyplot as plt
import pandas as pd

# Get cell coordinates
df_cells = pd.DataFrame(sdata_crop.tables["table_intensities"].obsm['spatial'], columns=['x', 'y'])

# Create figure and axis
fig, ax = plt.subplots(figsize=(10, 10))

# Plot cell density
h1 = ax.hexbin(
    df_cells["x"], df_cells["y"],
    gridsize=30,
    cmap="viridis",
    linewidths=0.2,
    edgecolors='face',
)
fig.colorbar(h1, ax=ax, label='Cell Count')
ax.set_title("Cell Density (Hexbin)")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.axis("equal")
ax.invert_yaxis()

plt.tight_layout()
plt.show()

### 4.4 Preprocessing

In [ ]:
sdata_crop = hp.tb.preprocess_proteomics(
    sdata_crop,
    labels_layer="labels_cells_instanseg_artifacts_filtered",
    table_layer="table_intensities",
    output_layer="table_intensities_preprocessed",
    size_norm=True,
    overwrite=True,
)

### 4.5 Leiden clustering

In [ ]:
import scanpy as sc

# Leiden clustering
sdata_crop = hp.tb.leiden(
    sdata_crop,
    labels_layer="labels_cells_instanseg_artifacts_filtered",
    table_layer="table_intensities_preprocessed",
    output_layer="table_intensities_leiden",
    calculate_umap=True,
    calculate_neighbors=True,
    n_pcs=17, # The number of principal components to use when calculating neighbors.
    n_neighbors=35, # The number of neighbors to consider when calculating neighbors.
    resolution=0.4,
    rank_genes=True,
    key_added="leiden",
    overwrite=True,
)

# Plot UMAP
sc.pl.umap(sdata_crop.tables["table_intensities_leiden"], color=["leiden"], show=True)

# Plot rank plot
sc.pl.rank_genes_groups(
    sdata_crop.tables["table_intensities_leiden"],
    n_genes=8,
    sharey=False,
    show=True,
)

In [ ]:
hp.pl.plot_shapes( 
    sdata_crop,
    img_layer="REAscreen_IO_CRC_image",
    table_layer="table_intensities_leiden",
    shapes_layer="shapes_cells_instanseg_artifacts_filtered",
    column="leiden",
    channel="DAPI (1)",
    linewidth=0.2,
    alpha=0.7,
    figsize=(8,8),
    to_coordinate_system="global",
)

In [ ]:
# We can plot other variables on the UMAP as well
from matplotlib.pyplot import rc_context
color_vars = [
    "area_cells",
    "contains_nucleus",
    "leiden",
    "CD11c",
    "CD326",
    "Actin",
]
with rc_context({"figure.figsize": (3, 3)}):
    sc.pl.umap(sdata_crop.tables["table_intensities_leiden"], color=color_vars, ncols=3)

In [ ]:
# Matrix plot
sc.pl.matrixplot(
    sdata_crop.tables["table_intensities_leiden"], 
    var_names=marker_channels, 
    groupby="leiden", 
    cmap="Blues",
    standard_scale="var",
    colorbar_title="column scaled\nexpression",
)

## 5. Ilastik object classification

In [ ]:
from ilastik.sdata_to_ilastik import export_h5, add_ilastik_to_sdata, assign_ilastik_cell_types, combine_ilastik_predictions

### 5.1 Export images for Ilastik

In [ ]:
# Paths to export h5 files
img_layer_to_export = "REAscreen_IO_CRC_image"
path_to_raw_h5 = os.path.join(output_path, f'raw_images.h5')

segmentation_mask_to_export = "labels_cells_instanseg_artifacts_filtered"
path_to_segmentation_h5 = os.path.join(output_path, 'segmentation_mask.h5')

# Export raw image channels
export_h5(
    sdata=sdata_crop, 
    img_layer=img_layer_to_export, 
    channels = None, # Will export all channels
    output=path_to_raw_h5, 
    crd=None # Images will not be cropped
)

# Export segmentation mask
export_h5(
    sdata=sdata_crop, 
    labels_layer=segmentation_mask_to_export, 
    output=path_to_segmentation_h5, 
    crd=None
)

# NOTE: To avoid crashing Ilastik, the images should not be too large.

### 5.2 Training Ilastik object classifiers

There are a variety of diffent object classifiers that can be created in ilastik.

Some options:
- You can train on a single image or on a combination of images (but more images will increase complexity and computation time)
- You can train on any set of features that is most appropriate (but more features will increase complexity and computation time)
- You can have as many classes as is needed (but more classes will increase complexity and computation time)
- As a rule of thumb, you can classify for anything as long as you can visually recognize it yourself on an image (or a set of images)

For example:
- You can use the nucleus channel to classify cells in good segmentations and bad segmentations.
- You can train for each channel a classifier that classifies cells in positive and negative for that marker.
- You can train on multiple channels simultaneously to classify cells in different cell types.

For each object classifier, do the following:

Open Ilastik (v.1.4.0) and create an new project: Object Classification [Inputs: Raw Data, Segmentation]

Note that this documentation was written for version v.1.4.0, but everything seems to work with the latest version (v.1.4.1) as well.

**1. Input Data** </br>
To load in a separate channel, you need to do the following:

- Under the tab `Raw Data` from `1. Input Data`, you would click `Add...` and select `Add separate Image(s)...`.
- Select the .h5 file containing the raw images and you will need to specify which channel you want to work on from the drop-down menu and click `OK` (this specifies the internal path in the h5 file).

To load in multiple channels, you need to specify a correct pattern that also includes the internal path (i.e. in the h5 file) to the correct images.

For example, to combine channel_1 and channel_2, you would do the following steps:
- Under the tab 'Raw Data' from '1. Input Data', you would click 'Add...' and select 'Add a single 3D/4D Volume from Sequence...'.
- Under `Specify Pattern`, you enter a patterns that specifies the images of interest. For example: `/path/to/images/raw_images.h5/channel_1; /path/to/images/raw_images.h5/channel_2` and click `Apply`. It is important that the path is specified correctly, multiple paths should be separated by a semicolon and the internal path in the h5 file should be specified as well in the path.
- Select `Stack Across: C` before clicking `OK`.

You will also need to add the segmentation mask. Under the tab `Segmentation Image`, you click `Add...` and select `Add separate Image(s)...`. You then select the appropriate segmentation mask file (e.g. `segmentation_mask.h5`).

After adding the Raw Data and the Segmentation Image to the ilastik project, it is useful to right-click on them, go to `Edit properties...` and make sure `Storage:` is set to `Copy into project file`. This makes sure, you can move around the ilastik project file (on your computer or even to other computers) without losing the link to the files the project was trained on (and risk losing your training). It is also useful to set `Nickname:` to something informative such as the name of the tissue/sample/replicate/etc (to keep track of which image is which). By default, this will be set to the filename.

<b>Important:</b> Note that the images for training in the Ilastik project can't be too large or Ilastik will crash. Around 10000 x 10000 pixels seems to work fine, but smaller (for training at least) is preferred. 

**2. Object Feature Selection** </br>
To select features for training, it would be recommended to not select all of them, but be mindful of which you want to train on. Although ilastik itself describes computing many features at once as computationally cheap, it can still really add up to calculate all features if there are a lot of cells in each image. Additionally, by, for example, only working on intensity-related features, it becomes more explainable what the model was trained on how it can be interpreted.

For most cases, we would recommend to follow these steps:
- Under the tab `2. Object Feature Selection`, click `Select Features` and click all boxes under `Intensity Distribution`. You can add other features that you know are relevant, if needed (such as `Size in pixels` or `Diameter`). In case you want create a classifier to distinguish good segmentations from bad segmentation, it would make sense to select all features.
- After clicking `OK`, wait until all features have been computed before moving on to the next step.

<b>Important:</b> Adding too many features (i.e. more than is necessary to get good classification) risks crashing the Ilastik project. This can also create problems for large images when running headless mode. Working on only the intensity features seems to work fine.

**3. Object Classification** </br>
Here, you can create multiple classes for your classifier and train them until you are satisfied with the results. In general, it is useful to initially add a good amount of labels for the different classes for different regions of the image (or even already over multiple images) to capture the variation that is in the data and click `Live Update` to see the prediction results. Subsequently, you can focus more on the mistakes that are being made and add labels to correct those. When labeling, we suggest to unclick `Live Update` to avoid waiting time. It can be useful to use the `Uncertainty` layer to see which objects are still not robustly trained for. When training, ilastik does not make it easy to change the visualization, but if you right-click on `Raw Input` and select `Adjust Thresholds`, you have some options to change the display. When training on individual channels for positive/negative, preferably use the following format to name the classes: channel_pos and channel_neg (e.g.: CD45_pos and CD45_neg).

Note that you can save your training labels by right-clicking `Labels` and subsequently `Export...`. Set the `File` to where you want to save the labels and under what name. This entire process needs to be done for every image separately, but it can be worth it to be able to recreate the ilastik project if something goes wrong or if we would want to preprocess the images for example. Of course, this only works with the exact same segmentation masks (so not after changing segmentation settings or working on a different image crop).

**4. Object Information Export** </br>
<b>Important</b>: You need to click `Configure Feature Table Export` and change `Format` to `CSV (.csv)`. You also need to set these settings when you don't want to export the results from the GUI, but will use headless mode in, for example, a Jupyter notebook. The headless mode requires this setting to be set in the GUI so if you don't do it, you will get an error later on. 

If you do want to export the results for the training data, you can specify under `Choose File` where and under what name you want to export the results. By default, this is set to `{dataset_dir}/{nickname}.csv`, with `{dataset_dir}` refering to the directory the raw_images.h5 file is in and `{nickname}` refering to the name specified in the `1. Input Data` tab. Preferably, remove `{nickname}` and put a unique name in its place for each ilastik classifier. For example: for a classifier to label cells as positive/negative for channel_1, put `{dataset_dir}/channel_1.csv`.

Note that, by default, the object predictions will be saved as images as well, while these files will not be used in the subsequent analysis steps (since we will get all useful data from the csv files). Unfortunately, there is no way to avoid saving these files.

**5. Batch processing** </br>
We will not use the batch processing tab since it is not possible to specify the internal paths in the h5 files.

**6. Blockwise Object classification** </br>
<b>Important</b>: Before closing the project, you need to set the Blocks to, for example, 4000 x 4000 and the Halo to 100 x 100 and save the project. This is necessary for running the headless mode since it gets these settings from the Ilastik project and you can't set the settings in headless mode itself.

In [ ]:
# Specify paths for Ilastik object classifiers
ilastik_obj_classifiers = {
    'CD8a': { # Name of Ilastik object classifier. This will be used as an identifier for the classifier throughout the analysis 
        'path': r'd:\example_data_MACSima\REAscreen_IO_CRC\output_harpy\CD8a.ilp', # Path to Ilastik object classifier.
        'raw_images_internal_path_list': ['CD8a'], # Internal path in h5 file that was used during Ilastik training.
        'csv_path': output_path, # Path to where you manually exported the CSV from the GUI, or where you want the headless mode to export the CSV to.
        'headless': False # Whether to use headless mode to creat the CSV
    },
    'CD3': { # Name of Ilastik object classifier. This will be used as an identifier for the classifier throughout the analysis 
        'path': r'd:\example_data_MACSima\REAscreen_IO_CRC\output_harpy\CD3.ilp', # Path to Ilastik object classifier.
        'raw_images_internal_path_list': ['CD3'], # Internal path in h5 file that was used during Ilastik training.
        'csv_path': output_path, # Path to where you manually exported the CSV from the GUI, or where you want the headless mode to export the CSV to.
        'headless': False # Whether to use headless mode to creat the CSV
    },
}

### 5.3 OPTIONAL: Creating CSV-files using Ilastik headless mode
This is a useful option if you created an object classifier and you want to use it with data that is not in the training set. Headless mode will then use the existing model and create a CSV-file for the new dataset.

Headless mode is also very useful if your image is too large to reliably work with the Ilastik GUI. You can then train the classifier on a crop of the data and use headless mode to get the results for the entire image.

In [ ]:
import subprocess

# Specify path to Ilastik installation
path_to_ilastik_exe = r'c:\Program Files\ilastik-1.4.1.post1\ilastik.exe' # NOTE: Set this to the path to your Ilastik installation

# Run through all classifiers
for classifier_name, classifier_dict in ilastik_obj_classifiers.items():
    
    classifier_path = classifier_dict['path']
    raw_images_internal_path_list = classifier_dict['raw_images_internal_path_list']
    csv_path = classifier_dict['csv_path']
    headless = classifier_dict['headless']
    
    # Skip classifier if headless == False
    if not headless:
        print(f'skipping: {classifier_name}')
        continue
    
    # Expected output CSV filename (with '_table.csv' suffix)
    actual_csv_filename = os.path.join(csv_path, classifier_name + '_table.csv') # This is different from the '--table_filename' path because Ilastik always adds '_table' to the filename.
    
    # Check if output file already exists (possibly from exporting training data from the Ilastik GUI)
    if os.path.exists(actual_csv_filename):
        raise FileExistsError(f"{actual_csv_filename} already exists.")
    
    # Run ilastik headless as a subprocess
    cmd = [
        path_to_ilastik_exe,
        '--headless',
        '--project=' + classifier_path,
        '--raw_data=' + ";".join([f"{path_to_raw_h5}/{channel}" for channel in raw_images_internal_path_list]),
        '--segmentation_image=' + path_to_segmentation_h5 + segmentation_mask_to_export,
        '--output_format=' + 'png',
        '--export_source=' + "Blockwise Object Predictions", # Block size has to be set in the Ilastik project. We recommmend Blocks of 4000x4000 pixel and a Halo of 100x100 pixels.
        '--table_filename=' + os.path.join(csv_path, classifier_name + '.csv') # CSV has to be set as the export format in the Ilastik project.
    ]

    print(f'Running: {classifier_name}')
    result = subprocess.run(cmd, capture_output=True, text=True)

    ## OPTIONAL: Print the output and errors for debugging
    print('stdout:', result.stdout)
    print('stderr:', result.stderr)

    # Check if CSV file has been created
    if os.path.exists(actual_csv_filename):
        print(f"Output CSV file {actual_csv_filename} has been created successfully.")
    else:
        raise ValueError(f"Output CSV file {actual_csv_filename} was not created.")
    
# NOTE: 
#  - When running this cell, make sure to close the Ilastik GUI since the headless mode will give an error if one of the ilastik projects is opened.
#  - Ilastik is very optimistic and will always tell you "The operation completed successfully.", even when there was an error. This is why we check whether the file was actually created.

### 5.4 Adding Ilastik data back to SpatialData

In [ ]:
# Add data from all ilastik csv files in folder to sdata
for classifier_name, classifier_dict in ilastik_obj_classifiers.items():
    
    # Get path to CSV
    csv_path = classifier_dict['csv_path']
    actual_csv_filename = os.path.join(csv_path, classifier_name + '_table.csv')
    
    # Add ilastik results to sdata
    print('Running: ', classifier_name)
    add_ilastik_to_sdata(
        sdata = sdata_crop,
        input_path = actual_csv_filename,
        table_layer = 'table_intensities_leiden',
        labels_layer = segmentation_mask_to_export,
        centroid_column_x = 'centroid_x_cells',
        centroid_column_y = 'centroid_y_cells', 
        suffix = classifier_name
    )

# NOTE: This code will attempt to merge the data obtained from the ilastik classifiers to the sdata.tables[table_layer].obs based on the centroid coordinates of the sdata cells and the ilastik objects (i.e. for each cell in the sdata, the closest cell in the ilastik data will be considered a match). 
# The suffix parameter is used to add a unique identifier to all columns associated with each ilastik dataset.


In [ ]:
# Inspect table layer obs
sdata_crop.tables["table_intensities_leiden"].obs

In [ ]:
# Plot user labels
from matplotlib.colors import ListedColormap
cmap = ListedColormap(["#0082C8", "#848484", "#FFE119"])

hp.pl.plot_shapes(
    sdata_crop, 
    img_layer='REAscreen_IO_CRC_image', 
    channel='DAPI (1)', 
    shapes_layer='shapes_cells_instanseg_artifacts_filtered',
    table_layer='table_intensities_leiden',
    alpha=1,
    linewidth=0.6,
    cmap=cmap,
    column="ilastik_user_label_CD8a",
)
# There will be no user labels if you use headless mode

In [ ]:
# Plot predictions
cmap = ListedColormap(['#0082C8', "#FFE119"])

hp.pl.plot_shapes(
    sdata_crop, 
    img_layer='REAscreen_IO_CRC_image', 
    channel='DAPI (1)', 
    shapes_layer='shapes_cells_instanseg_artifacts_filtered',
    table_layer='table_intensities_leiden',
    alpha=1,
    linewidth=0.6,
    cmap=cmap,
    column="ilastik_predicted_class_CD8a",
)

In [ ]:
# Plot probabilities
hp.pl.plot_shapes(
    sdata_crop, 
    img_layer='REAscreen_IO_CRC_image', 
    channel='DAPI (1)', 
    shapes_layer='shapes_cells_instanseg_artifacts_filtered',
    table_layer='table_intensities_leiden',
    alpha=1,
    linewidth=0.6,
    cmap='coolwarm',
    column="ilastik_probability_pos_CD8a"
)

In [ ]:
# Plot probabilities
hp.pl.plot_shapes(
    sdata_crop, 
    img_layer='REAscreen_IO_CRC_image', 
    channel='DAPI (1)', 
    shapes_layer='shapes_cells_instanseg_artifacts_filtered',
    table_layer='table_intensities_leiden',
    alpha=1,
    linewidth=0.6,
    cmap='coolwarm',
    column="ilastik_probability_neg_CD8a"
)

### 5.5 Assigning cell types

In [ ]:
annotation_table_path = os.path.join(output_path, "annotation_matrix.csv")

sdata_crop = assign_ilastik_cell_types(
    sdata_crop, 
    annotation_table_path=annotation_table_path,
    table_layer='table_intensities_leiden',
    labels_layer='labels_cells_instanseg_artifacts_filtered', # The label layer that corresponds to the data in the table layer.
    output_column='ilastik_cell_types', # Name of output column
    default_value='other', # Values used for all cells that do not fit any of the conditions.
)

In [ ]:
# Plot cell types
hp.pl.plot_shapes(
    sdata_crop, 
    img_layer='REAscreen_IO_CRC_image', 
    channel='DAPI (1)', 
    shapes_layer='shapes_cells_instanseg_artifacts_filtered',
    table_layer='table_intensities_leiden',
    alpha=1,
    linewidth=0.6,
    cmap='rainbow',
    column="ilastik_cell_types",
)

## 6. Ilastik pixel classification
Creating pixel classifiers in Ilastik can be useful to semi-automatically annotate ROIs/artifacts/tissue.

### 6.1 Train pixel classifier
Open Ilastik (v.1.4.0) and create an new project: Pixel Classification.

Note that this documentation was written for version v.1.4.0, but everything seems to work with the latest version (v.1.4.1) as well.

**1. Input Data** </br> 
To load in a separate channel, you need to do the following:

- Under the tab `Raw Data` from `1. Input Data`, you would click `Add...` and select `Add separate Image(s)...`.
- Select the .h5 file containing the raw images and you will need to specify which channel you want to work on from the drop-down menu and click `OK` (this specifies the internal path in the h5 file).

To load in multiple channels, you need to specify a correct pattern that also includes the internal path (i.e. in the h5 file) to the correct images.

For example, to combine channel_1 and channel_2, you would do the following steps:
- Under the tab 'Raw Data' from '1. Input Data', you would click 'Add...' and select 'Add a single 3D/4D Volume from Sequence...'.
- Under `Specify Pattern`, you enter a patterns that specifies the images of interest. For example: `/path/to/images/raw_images.h5/channel_1; /path/to/images/raw_images.h5/channel_2` and click `Apply`. It is important that the path is specified correctly, multiple paths should be separated by a semicolon and the internal path in the h5 file should be specified as well in the path.
- Select `Stack Across: C` before clicking `OK`.

After adding the Raw Data to the ilastik project, it is useful to right-click on them, go to `Edit properties...` and make sure `Storage:` is set to `Copy into project file`. This makes sure, you can move around the ilastik project file (on your computer or even to other computers) without losing the link to the files the project was trained on (and risk losing your training). It is also useful to set `Nickname:` to something informative such as the name of the tissue/sample/replicate/etc (to keep track of which image is which). By default, this will be set to the filename.

<b>Important:</b> Note that the images for training in the Ilastik project can't be too large or Ilastik will crash. Around 10000 x 10000 pixels seems to work fine, but smaller (for training at least) is preferred. 

**2. Feature Selection** </br>
- Click `Select Features...`
- Select all features
- After clicking `OK`, wait until all features have been computed before moving on to the next step.

**3. Training** </br>
Here, you can create multiple classes for your classifier and train them until you are satisfied with the results. In general, it is useful to initially add a good amount of labels for the different classes for different regions of the image (or even already over multiple images) to capture the variation that is in the data and click `Live Update` to see the prediction results. Subsequently, you can focus more on the mistakes that are being made and add labels to correct those. When labeling, we suggest to unclick `Live Update` to avoid waiting time. It can be useful to use the `Uncertainty` layer to see which pixels are still not robustly trained for. When training, ilastik does not make it easy to change the visualization, but if you right-click on `Raw Input` and select `Adjust Thresholds`, you have some options to change the display.

Note that you can save your training labels by right-clicking `Labels` and subsequently `Export...`. Set the `File` to where you want to save the labels and under what name. This entire process needs to be done for every image separately, but it can be worth it to be able to recreate the ilastik project if something goes wrong or if we would want to preprocess the images for example.

**4. Prediction Export** </br>
- Under `Source`, we have a couple of options to export that can be useful, but we will select `Simple Segmentation` for now. 
- Click `Choose Export Image Settings...` and set Format to `tiff`. Change the File name to something informative and set `Convert to Data Type` to `integer 8-bit`.
- Click `OK` and click `Export All` to export the segmentation mask

**5. Batch processing** </br>
We will not use the batch processing tab since it is not possible to specify the internal paths in the h5 files.

### 6.2 Add results to SpatialData

In [ ]:
# Read in segmentation mask and clean up mask
from skimage import io, morphology

mask = io.imread(os.path.join(output_path, 'ROI_segmentation.tiff'))
mask = (mask == 1) # Keep only class 1 pixels
mask = morphology.remove_small_objects(mask, min_size=15000)
mask = morphology.remove_small_holes(mask, area_threshold=15000)
mask = mask.astype(np.uint8) # Convert boolean mask back to integers

sdata_crop = hp.im.add_labels_layer(
    sdata_crop, 
    arr=mask, 
    output_layer="ilastik_mask", 
    chunks=1024, 
    overwrite=True
)

In [ ]:
# Plot mask
sdata_crop.pl.render_labels(
    "ilastik_mask", 
).pl.show(
    coordinate_systems="global",
    figsize=(10, 10)
)

## 7. Superpixel clustering

In [ ]:
# First, we create new labels and shapes layers for a hexagonal grid 
import xarray
xa: xarray.DataArray = sdata.images['REAscreen_IO_CRC_image'][f'scale0']['image']
shape = (xa.shape[1], xa.shape[2])

size = 50 # radius of the hexagon, or size length of the square.

sdata = hp.im.add_grid_labels_layer(
    sdata, 
    shape=shape, 
    size=size, 
    output_shapes_layer=f"shapes_spots_{size}um", 
    output_labels_layer=f"labels_spots_{size}um", 
    grid_type='hexagon', # Set to 'square' for square grid
    offset=(0, 0), 
    chunks=1024, 
    client=None, 
    transformations=None, 
    scale_factors=(2, 2, 2, 2),
    overwrite=True
)

In [ ]:
# Plot the grid
hp.pl.plot_shapes( 
    sdata,
    img_layer="REAscreen_IO_CRC_image",
    shapes_layer=f"shapes_spots_{size}um",
    channel="DAPI (1)",
    linewidth=0.4,
    alpha=0.3,
    figsize=(14,10),
    to_coordinate_system="global",
)

In [ ]:
# Allocate intensities 
sdata = hp.tb.allocate_intensity( 
    sdata,
    img_layer= "REAscreen_IO_CRC_image",
    labels_layer=f"labels_spots_{size}um",
    output_layer="table_superpixel_intensities",
    channels=marker_channels,
    mode="sum", # Can be set to "mean" or "sum"
    to_coordinate_system="global",
    overwrite=True,
)

In [ ]:
# Preprocess 
sdata=hp.tb.preprocess_proteomics(
    sdata,
    labels_layer=f"labels_spots_{size}um",
    table_layer="table_superpixel_intensities",
    output_layer="table_superpixel_intensities_prepocessed",
    size_norm=True,
    log1p=True,
    scale=False,
    max_value_scale=10,
    q=None,
    calculate_pca=False,
    n_comps=50,
    overwrite=True,
)

In [ ]:
import scanpy as sc

# Leiden clustering
sdata = hp.tb.leiden(
    sdata,
    labels_layer=f"labels_spots_{size}um",
    table_layer="table_superpixel_intensities_prepocessed",
    output_layer="table_superpixel_intensities_leiden",
    calculate_umap=True,
    calculate_neighbors=True,
    n_pcs=17, # The number of principal components to use when calculating neighbors.
    n_neighbors=35, # The number of neighbors to consider when calculating neighbors.
    resolution=0.4,
    rank_genes=True,
    key_added="leiden",
    overwrite=True,
)

# Plot UMAP
sc.pl.umap(sdata.tables["table_superpixel_intensities_leiden"], color=["leiden"], show=True)

# Plot rank plot
sc.pl.rank_genes_groups(
    sdata.tables["table_superpixel_intensities_leiden"],
    n_genes=8,
    sharey=False,
    show=True,
    )

In [ ]:
# Plot Leiden clusters spatially
hp.pl.plot_shapes( 
    sdata,
    img_layer="REAscreen_IO_CRC_image",
    table_layer="table_superpixel_intensities_leiden",
    shapes_layer=f"shapes_spots_{size}um",
    column="leiden",
    cmap="rainbow",
    channel="DAPI (1)",
    linewidth=0.2,
    alpha=0.7,
    figsize=(14,10),
    to_coordinate_system="global",
)

In [ ]:
# Plot expression of CD11c
hp.pl.plot_shapes( 
    sdata,
    img_layer="REAscreen_IO_CRC_image",
    table_layer="table_superpixel_intensities_leiden",
    shapes_layer=f"shapes_spots_{size}um",
    column="CD11c",
    channel="DAPI (1)",
    linewidth=0.2,
    alpha=0.7,
    figsize=(14,10),
    to_coordinate_system="global",
)

In [ ]:
# Matrix plot
sc.pl.matrixplot(
    sdata.tables["table_superpixel_intensities_leiden"], 
    var_names=marker_channels, 
    groupby="leiden", 
    cmap="Blues",
    standard_scale="var",
    colorbar_title="column scaled\nexpression",
)

## 8. Pixel clustering (FlowSOM)

In [ ]:
# First we create a crop to speed up the computation time for the workshop
from spatialdata import bounding_box_query

se = bounding_box_query(
    sdata["REAscreen_IO_CRC_image"],
    axes=("y", "x"),
    min_coordinate=[0, 5000], # y_min, x_min
    max_coordinate=[5000, 10000], # y_max, x_max
    target_coordinate_system="global",
)

sdata.images["REAscreen_IO_CRC_image_flowsom_crop"] = se
sdata.write_element(
    "REAscreen_IO_CRC_image_flowsom_crop", overwrite=True
)

In [ ]:
# Plot ROI layer on crop
hp.pl.plot_shapes(
    sdata_crop, 
    img_layer='REAscreen_IO_CRC_image', 
    channel='DAPI (1)', 
    shapes_layer='ROI',
)

### 8.1 Preprocessing

In [ ]:
# The preprocessing step normalizes and blurs the images based on various quantile and gaussian blur parameters
sdata_crop = hp.im.pixel_clustering_preprocess(
    sdata_crop,
    img_layer="REAscreen_IO_CRC_image",
    output_layer="REAscreen_IO_CRC_image_flowsom_preprocessed",
    channels=successful_channels,
    q=99, # Quantile used for normalization. Each channel is normalized by its own calculated quantile.
    q_sum=5, # If the sum of the channel values at a pixel is below this quantile, the pixel values across all channels are set to NaN.
    q_post=99.9, # Quantile used for normalization after other preprocessing steps (`q`, `q_sum`, `norm_sum` normalization and Gaussian blurring) are performed.
    sigma=2, # Gaussian blur parameter for each channel. Use `0` to omit blurring for specific channels or `None` to skip blurring altogether.
    norm_sum=True, # If `True`, each channel is normalized by the sum of all channels at each pixel.
    cap_max=1, # The maximum allowable value for the elements in the resulting preprocessed image layers. If `None`, no capping is applied. Typical value would be `1.0` to exclude outliers.
    overwrite=True,
)

In [ ]:
# Let's compare the histogram for CD4 after preprocessing with the original image
channel = "CD4"

hp.pl.histogram(
    sdata_crop,
    img_layer="REAscreen_IO_CRC_image",
    channel=channel,
    bins=100,
    fig_kwargs={
        "figsize": (4, 4),
    },
)

hp.pl.histogram(
    sdata_crop,
    img_layer="REAscreen_IO_CRC_image_flowsom_preprocessed",
    channel=channel,
    bins=100,
    fig_kwargs={
        "figsize": (4, 4),
    },
)

### 8.2 FlowSOM pixel clustering

In [ ]:
# FlowSOM clustering

import flowsom as fs
from dask.distributed import Client, LocalCluster

work_with_client = False

if work_with_client:
    # client example
    cluster = LocalCluster(
        n_workers=1,
        threads_per_worker=10,
    )

    client = Client(cluster)
else:
    client = None

batch_model = fs.models.BatchFlowSOMEstimator

sdata_crop, fsom, mapping = hp.im.flowsom(
    sdata_crop,
    img_layer=["REAscreen_IO_CRC_image_flowsom_preprocessed"],
    output_layer_clusters=[
        "flowsom_clusters",
    ],  # we need output_cluster_layer and output_meta_cluster_layer --> these will both be labels layers
    output_layer_metaclusters=[
        "flowsom_metaclusters",
    ],
    channels = successful_channels, 
    fraction = 0.1, # Fraction of the data to sample for training flowsom.
    n_clusters=20, # The number of meta clusters to form.
    random_state=111,
    chunks=512,
    client=client,
    model=batch_model,
    num_batches=10,
    xdim=10,
    ydim=10,
    z_score=False,
    z_cap=3,
    persist_intermediate=True,
    overwrite=True,
)

In [ ]:
# Plot all FlowSOM clusters
hp.pl.pixel_clusters(
    sdata_crop,
    labels_layer="flowsom_clusters",
    figsize=(14, 10),
    to_coordinate_system="global",
    render_labels_kwargs={"alpha": 1, "cmap": "rainbow", "to_coordinate_system": "global"},
    coordinate_systems="global", # passed to .pl.show()
)

# Plot FlowSOM metaclusters
hp.pl.pixel_clusters(
    sdata_crop,
    labels_layer="flowsom_metaclusters",
    figsize=(14, 10),
    to_coordinate_system="global",
    render_labels_kwargs={"alpha": 1, "cmap": "rainbow", "to_coordinate_system": "global"},
    coordinate_systems="global", # passed to .pl.show()
)

In [ ]:
# Calculate average intensities per SOM cluster
sdata_crop = hp.tb.cluster_intensity( # This function computes average intensity for each SOM cluster identified in the `labels_layer` and stores the results in a new table layer (`output_layer`).
    sdata_crop,
    mapping=mapping,
    img_layer=["REAscreen_IO_CRC_image_flowsom_preprocessed"],
    labels_layer=["flowsom_clusters"],
    to_coordinate_system=["global"],
    output_layer="counts_clusters",
    overwrite=True,
)

In [ ]:
# Plot heatmaps for FlowSOM clusters and metaclusters
for _metaclusters in [True]:
    hp.pl.pixel_clusters_heatmap(
        sdata_crop,
        table_layer="counts_clusters",
        figsize=(40, 16),
        fig_kwargs={"dpi": 300},
        linewidths=0.001,
        metaclusters=_metaclusters,
        z_score=True,
        output='img/FlowSOM_full_image_heatmap.png'
    )

We ran FlowSOM pixel clustering on the entire image using the same settings and n_clusters set to 15. You can find the results below:

![Alt text](img/FlowSOM_full_image_spatial.png)

### 8.3 Spatial enrichment FlowSOM metaclusters 

We can use hp.tb.spatial_pixel_neighbors() after calculating FlowSOM metaclusters to get neighborhood enrichment scores among metaclusters. This function extracts grid-based cluster labels from the specified labels layer of a SpatialData object,
subdivides the spatial domain into a grid using a specified sampling interval, and computes spatial neighbors along with
neighborhood enrichment statistics. The resulting AnnData object stores the cluster labels as a categorical
observation (under the key provided by `key_added`) and the corresponding spatial coordinates in its `.obsm`
attribute. `squidpy` is used for the spatial neighbors computation and
the neighborhood enrichment analysis (i.e. `squidpy.gr.spatial_neighbors` and `squidpy.gr.nhood_enrichment`).
Results can then be visualized using e.g. `squidpy.pl.nhood_enrichment`.

In [ ]:
import numpy as np
import squidpy as sq

key_added = "cluster_id"

adata = hp.tb.spatial_pixel_neighbors(
    sdata_crop,
    labels_layer="flowsom_metaclusters",
    key_added=key_added,
    mode="most_frequent",
    grid_type="hexagon",
    size=20,
    subset=None,
)

adata.uns[f"{key_added}_nhood_enrichment"]["zscore"] = np.nan_to_num(
    adata.uns[f"{key_added}_nhood_enrichment"]["zscore"]
)
sq.pl.nhood_enrichment(adata, cluster_key=key_added, method="ward", mode="zscore", figsize=(8, 8))

# NOTE: cluster_id 0 contains all pixels that were filtered out during preprocessing

## 9. Cell clustering (FlowSOM)

In [ ]:
# Assign cell types based on FlowSOM pixel metaclusters

batch_model = fs.models.BatchFlowSOMEstimator

sdata_crop, fsom = hp.tb.flowsom(
    sdata_crop,
    labels_layer_cells=["labels_cells_instanseg_artifacts_filtered"],
    labels_layer_clusters=["flowsom_metaclusters"],
    output_layer="table_cell_clustering_flowsom",
    n_clusters=15,
    chunks=512,
    model=batch_model,
    num_batches=10,
    random_state=100,
    overwrite=True,
)

In [ ]:
# Inspect table layer
sdata_crop.tables['table_cell_clustering_flowsom']

In [ ]:
# Inspect obs
sdata_crop.tables['table_cell_clustering_flowsom'].obs

In [ ]:
# Plot cells colored by FlowSOM cluster
sdata_crop.pl.render_labels(
    "labels_cells_instanseg_artifacts_filtered", 
    table_name="table_cell_clustering_flowsom", 
    color="clustering", 
    cmap="rainbow"
).pl.show(
    coordinate_systems="global",
    figsize=(10, 10)
)

In [ ]:
# Plot cells colored by FlowSOM metacluster
sdata_crop.pl.render_labels(
    "labels_cells_instanseg_artifacts_filtered", 
    table_name="table_cell_clustering_flowsom", 
    color="metaclustering", 
    cmap="rainbow"
).pl.show(
    coordinate_systems="global",
    figsize=(10, 10)
)